In [1]:
import tensorflow as tf
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
from nltk.tokenize.casual import TweetTokenizer
from w266_common import utils, vocabulary
import time
import datetime
import os


/home/saulgrimaldo1/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import csv
tokenizer = TweetTokenizer()
x_data = []
x_contexts = []
labels = []
sentences  = []
contexts = []
with open('merged_data_v4.csv', 'r') as csvfile:
    linereader = csv.reader(csvfile, delimiter = '|')
    for i, row in enumerate(linereader):
        if i == 0:
            continue
        sentence, context, sarcasm = row
        sentences.append(sentence)
        contexts.append(context)
        x_tokens = utils.canonicalize_words(tokenizer.tokenize(sentence), hashtags = True)
        context_tokens = utils.canonicalize_words(tokenizer.tokenize(context), hashtags = True)
        x_data.append(x_tokens)
        x_contexts.append(context_tokens)
        labels.append(int(sarcasm))


#rng = np.random.RandomState(5)
#rng.shuffle(x_data)  # in-place
#train_split_idx = int(0.7 * len(labels))
#test_split_idx  = int(0.9 * len(labels))

shuffle_indices = np.random.permutation(np.arange(len(labels)))
train_split_idx = int(0.7 * len(labels))
test_split_idx  = int(0.9 * len(labels))

train_indices = shuffle_indices[:train_split_idx]
validation_indices = shuffle_indices[train_split_idx:test_split_idx]
test_indices = shuffle_indices[test_split_idx:]


train_sentences = np.array(x_data)[train_indices]
train_contexts = np.array(x_contexts)[train_indices]
train_labels= np.array(labels)[train_indices] 
validation_sentences = np.array(x_data)[validation_indices]
validation_labels = np.array(labels)[validation_indices]
validation_contexts = train_contexts = np.array(x_contexts)[validation_indices]
test_sentences = np.array(x_data)[test_indices]  
test_contexts = train_contexts = np.array(x_contexts)[test_indices]
test_labels = np.array(labels)[test_indices]  

In [3]:
a = [2]*4
a[2] = 6
a

[2, 2, 6, 2]

In [4]:
def transform_labels(raw_label_set, size):
    label_set = []
    for label in raw_label_set:
        labels = [0] * size
        labels[label] = 1
        label_set.append(labels)
    return np.array(label_set)

expanded_train_labels = transform_labels(train_labels, 2)
expanded_validation_labels = transform_labels(validation_labels,2)
expanded_test_labels = transform_labels(test_labels,2)

In [5]:
a = [1,3,4,5,6,7,8,8,1,5,6,7]
a + ["<PADDING>"]*5

[1,
 3,
 4,
 5,
 6,
 7,
 8,
 8,
 1,
 5,
 6,
 7,
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>']

In [6]:
class PaddingAndTruncating:
    def __init__(self, max_len):
        self.max_len = max_len
    
    def pad_or_truncate(self, sentence):
        sen_len = len(sentence)
        paddings = self.max_len - sen_len
        if paddings >=0:
            return list(sentence) + ["<PADDING>"] * paddings
        return sentence[0:paddings]
        
PadAndTrunc = PaddingAndTruncating(10)
        
        
PadAndTrunc.pad_or_truncate(["the","angry","man","is","angry"])

['the',
 'angry',
 'man',
 'is',
 'angry',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>']

In [7]:
from tensorflow.contrib import learn
vocab_size = 5000

#max_len = max([len(sent) for sent  in train_sentences])
PadAndTrunc =PaddingAndTruncating(50)
train_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, train_sentences))
train_context_padded = list(map(PadAndTrunc.pad_or_truncate, train_contexts))
validation_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, validation_sentences))
validation_context_padded = list(map(PadAndTrunc.pad_or_truncate, validation_contexts))
test_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, test_sentences))
test_context_padded = list(map(PadAndTrunc.pad_or_truncate, test_contexts))

vocab = vocabulary.Vocabulary(utils.flatten(list(train_sentences_padded) + list(train_context_padded)), vocab_size)
train_s = np.array(list(map(vocab.words_to_ids, train_sentences_padded)))
train_c = np.array(list(map(vocab.words_to_ids, train_context_padded)))
validation_s = np.array(list(map(vocab.words_to_ids, validation_sentences_padded)))
validation_c = np.array(list(map(vocab.words_to_ids, validation_context_padded)))
test_s = np.array(list(map(vocab.words_to_ids, test_sentences_padded)))
test_c = np.array(list(map(vocab.words_to_ids, test_context_padded)))
train_s

array([[  12,    5,    9, ...,    3,    3,    3],
       [  12,    5,    9, ...,    3,    3,    3],
       [   8,    7,  482, ...,    3,    3,    3],
       ..., 
       [   5,  481,  148, ..., 1656,   92, 3098],
       [   8,  420,   18, ...,    3,    3,    3],
       [ 189,   19,    2, ...,    3,    3,    3]])

In [8]:
i = 3
("conv-maxpool-%s" % i)

'conv-maxpool-3'

In [9]:
class TextCNN(object):

    def __init__(
      self, sequence_length, num_classes, vocab_size, 
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x1 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x1")
        self.input_x2 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x2")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars1 = tf.nn.embedding_lookup(self.W, self.input_x1)
            self.embedded_chars_expanded1 = tf.expand_dims(self.embedded_chars1, -1)
            self.embedded_chars2 = tf.nn.embedding_lookup(self.W, self.input_x2)
            self.embedded_chars_expanded2 = tf.expand_dims(self.embedded_chars2, -1)

        # Create a convolution + avgpool layer for each filter size
        pooled_outputs = []

        for i, filter_size in enumerate(filter_sizes):
            with tf.variable_scope("conv-avgpool-%s" % i) as scope:
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]

                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
              
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded1,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h1 = tf.nn.tanh(tf.nn.bias_add(conv, b), name="tanh1")
               
                # Maxpooling over the outputs
                pooled = tf.nn.avg_pool(
                    h1,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                
                pooled_outputs.append(pooled)
                #pooled_outputs.append(pooled)
                scope.reuse_variables()
                 
                
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded2,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h2 = tf.nn.tanh(tf.nn.bias_add(conv, b), name="tanh2")
                # Maxpooling over the outputs
                pooled2 = tf.nn.avg_pool(
                    h2,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled2)

        # Combine all the pooled features

        num_filters_total = num_filters * len(filter_sizes) * 2

        self.h_pool = tf.concat(pooled_outputs, 1)

        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])


        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
        
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
           
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            self.labels = tf.argmax(self.input_y, 1)
            TP = tf.count_nonzero(self.predictions * self.labels)
            TN = tf.count_nonzero((self.predictions - 1) * (self.labels - 1))
            FP = tf.count_nonzero(self.predictions * (self.labels - 1))
            FN = tf.count_nonzero((self.predictions - 1) * self.labels)
            correct_predictions = tf.equal(self.predictions, self.labels)
            self.precision = TP / (TP + FP)
            self.recall = TP / (TP + FN)
            self.f1_score = 2 * self.precision * self.recall / (self.precision + self.recall)
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [ ]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [ ]:
learning_rate = 0.1

tf.flags.DEFINE_float("dev_sample_percentage", 0.75, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 100, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "1", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters",100, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.25, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0, "L2 regularization lambda (default: 0.0)")

# Training parameters
batch_size = 50
tf.flags.DEFINE_integer("batch_size", batch_size, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 250, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 20, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================


with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():

        cnn = TextCNN(
            sequence_length=train_s.shape[1],
            num_classes=2,
            vocab_size=vocab_size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)
        
         # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdagradOptimizer(learning_rate)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        #vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch1, x_batch2, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch1, x_batch2, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            #r 
            step, summaries, loss, accuracy, recall, precision, f1 = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy,cnn.recall, cnn.precision, cnn.f1_score],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}, rec {:g}, pre {:g}, f1 {:g}".format(time_str, step, loss, accuracy, recall, precision, f1))# recall, precision, f1))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(train_s, train_c,  expanded_train_labels)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch1, x_batch2, y_batch = zip(*batch)
            train_step(x_batch1, x_batch1, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(validation_s, validation_c, expanded_validation_labels, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))
print("\nTest Set:")
dev_step(test_s, test_c, expanded_test_labels, writer=dev_summary_writer)
print("")



Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=50
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.75
DROPOUT_KEEP_PROB=0.25
EMBEDDING_DIM=100
EVALUATE_EVERY=20
FILTER_SIZES=1
L2_REG_LAMBDA=0
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.neg
NUM_CHECKPOINTS=5
NUM_EPOCHS=250
NUM_FILTERS=100
POSITIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.pos

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-avgpool-0/W:0/grad/hist is illegal; using conv-avgpool-0/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-avgpool-0/W:0/grad/sparsity is illegal; using conv-avgpool-0/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-avgpool-0/b:0/grad/hist is illegal; using conv-avgpool-0/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-avgpool-0/b:0/gr

2018-04-21T23:48:18.604498: step 102, loss 0.685514, acc 0.64
2018-04-21T23:48:18.696483: step 103, loss 0.608003, acc 0.64
2018-04-21T23:48:18.792499: step 104, loss 0.617203, acc 0.78
2018-04-21T23:48:18.888490: step 105, loss 0.618681, acc 0.62
2018-04-21T23:48:18.983062: step 106, loss 0.607676, acc 0.6
2018-04-21T23:48:19.080491: step 107, loss 0.507033, acc 0.82
2018-04-21T23:48:19.177124: step 108, loss 0.583077, acc 0.72
2018-04-21T23:48:19.276535: step 109, loss 0.651711, acc 0.66
2018-04-21T23:48:19.372531: step 110, loss 0.637115, acc 0.6
2018-04-21T23:48:19.468728: step 111, loss 0.605654, acc 0.76
2018-04-21T23:48:19.568520: step 112, loss 0.728229, acc 0.72
2018-04-21T23:48:19.661396: step 113, loss 0.552143, acc 0.78
2018-04-21T23:48:19.760500: step 114, loss 0.490597, acc 0.8
2018-04-21T23:48:19.852746: step 115, loss 0.621159, acc 0.62
2018-04-21T23:48:19.944498: step 116, loss 0.518417, acc 0.7
2018-04-21T23:48:20.044485: step 117, loss 0.6742, acc 0.64
2018-04-21T23:

2018-04-21T23:48:34.264628: step 225, loss 0.537436, acc 0.7
2018-04-21T23:48:34.308091: step 226, loss 0.639389, acc 0.62
2018-04-21T23:48:34.351804: step 227, loss 0.704875, acc 0.54
2018-04-21T23:48:34.395246: step 228, loss 0.675314, acc 0.58
2018-04-21T23:48:34.438404: step 229, loss 0.512865, acc 0.88
2018-04-21T23:48:34.484588: step 230, loss 0.562706, acc 0.74
2018-04-21T23:48:34.527868: step 231, loss 0.61729, acc 0.64
2018-04-21T23:48:34.571901: step 232, loss 0.477181, acc 0.78
2018-04-21T23:48:34.616148: step 233, loss 0.567754, acc 0.74
2018-04-21T23:48:34.659279: step 234, loss 0.573118, acc 0.7
2018-04-21T23:48:34.705866: step 235, loss 0.602841, acc 0.72
2018-04-21T23:48:34.751577: step 236, loss 0.538314, acc 0.72
2018-04-21T23:48:34.795378: step 237, loss 0.516621, acc 0.72
2018-04-21T23:48:34.839829: step 238, loss 0.476352, acc 0.82
2018-04-21T23:48:34.883124: step 239, loss 0.508763, acc 0.74
2018-04-21T23:48:34.929844: step 240, loss 0.861112, acc 0.7

Evaluation:

2018-04-21T23:48:43.619508: step 345, loss 0.573276, acc 0.7
2018-04-21T23:48:43.663066: step 346, loss 0.515986, acc 0.7
2018-04-21T23:48:43.707110: step 347, loss 0.540903, acc 0.76
2018-04-21T23:48:43.750805: step 348, loss 0.545166, acc 0.82
2018-04-21T23:48:43.797066: step 349, loss 0.688068, acc 0.64
2018-04-21T23:48:43.845431: step 350, loss 0.569397, acc 0.66
2018-04-21T23:48:43.894866: step 351, loss 0.482328, acc 0.72
2018-04-21T23:48:43.939313: step 352, loss 0.497944, acc 0.78
2018-04-21T23:48:43.984392: step 353, loss 0.97218, acc 0.68
2018-04-21T23:48:44.029844: step 354, loss 0.481709, acc 0.84
2018-04-21T23:48:44.076510: step 355, loss 0.414028, acc 0.86
2018-04-21T23:48:44.120410: step 356, loss 0.413092, acc 0.88
2018-04-21T23:48:44.165968: step 357, loss 0.673572, acc 0.52
2018-04-21T23:48:44.209461: step 358, loss 0.682361, acc 0.6
2018-04-21T23:48:44.253402: step 359, loss 0.61085, acc 0.58
2018-04-21T23:48:44.299959: step 360, loss 0.563155, acc 0.68

Evaluation:


2018-04-21T23:48:56.902444: step 465, loss 0.517894, acc 0.64
2018-04-21T23:48:56.947951: step 466, loss 0.632382, acc 0.62
2018-04-21T23:48:56.994560: step 467, loss 0.551274, acc 0.66
2018-04-21T23:48:57.040160: step 468, loss 0.474823, acc 0.76
2018-04-21T23:48:57.085025: step 469, loss 0.381833, acc 0.82
2018-04-21T23:48:57.133890: step 470, loss 0.356279, acc 0.88
2018-04-21T23:48:57.179304: step 471, loss 0.380205, acc 0.86
2018-04-21T23:48:57.225901: step 472, loss 0.58153, acc 0.76
2018-04-21T23:48:57.271331: step 473, loss 0.393064, acc 0.82
2018-04-21T23:48:57.317606: step 474, loss 0.347937, acc 0.88
2018-04-21T23:48:57.369565: step 475, loss 0.511648, acc 0.74
2018-04-21T23:48:57.415430: step 476, loss 0.627592, acc 0.78
2018-04-21T23:48:57.460049: step 477, loss 0.547655, acc 0.8
2018-04-21T23:48:57.505000: step 478, loss 0.459316, acc 0.86
2018-04-21T23:48:57.551695: step 479, loss 0.862966, acc 0.76
2018-04-21T23:48:57.601414: step 480, loss 0.683533, acc 0.66

Evaluatio

...

2018-04-21T23:52:54.175695: step 3345, loss 0.174568, acc 0.92
2018-04-21T23:52:54.221751: step 3346, loss 0.229711, acc 0.92
2018-04-21T23:52:54.267038: step 3347, loss 0.255207, acc 0.94
2018-04-21T23:52:54.313546: step 3348, loss 0.187369, acc 0.94
2018-04-21T23:52:54.359249: step 3349, loss 0.151009, acc 0.94
2018-04-21T23:52:54.408663: step 3350, loss 0.216136, acc 0.9
2018-04-21T23:52:54.454573: step 3351, loss 0.39261, acc 0.84
2018-04-21T23:52:54.499232: step 3352, loss 0.346246, acc 0.84
2018-04-21T23:52:54.545271: step 3353, loss 0.150834, acc 0.98
2018-04-21T23:52:54.591022: step 3354, loss 0.169093, acc 0.96
2018-04-21T23:52:54.640620: step 3355, loss 0.217908, acc 0.92
2018-04-21T23:52:54.687477: step 3356, loss 0.145235, acc 0.94
2018-04-21T23:52:54.735008: step 3357, loss 0.228152, acc 0.9
2018-04-21T23:52:54.780102: step 3358, loss 0.364416, acc 0.96
2018-04-21T23:52:54.825139: step 3359, loss 0.19286, acc 0.94
2018-04-21T23:52:54.873953: step 3360, loss 0.169277, acc 0

2018-04-21T23:53:03.772738: step 3465, loss 0.116823, acc 0.98
2018-04-21T23:53:03.819379: step 3466, loss 0.192296, acc 0.94
2018-04-21T23:53:03.865527: step 3467, loss 0.246374, acc 0.88
2018-04-21T23:53:03.911200: step 3468, loss 0.215536, acc 0.92
2018-04-21T23:53:03.957786: step 3469, loss 0.220923, acc 0.9
2018-04-21T23:53:04.007945: step 3470, loss 0.136651, acc 0.96
2018-04-21T23:53:04.054601: step 3471, loss 0.231051, acc 0.92
2018-04-21T23:53:04.101027: step 3472, loss 0.114795, acc 0.96
2018-04-21T23:53:04.146622: step 3473, loss 0.162539, acc 0.94
2018-04-21T23:53:04.192093: step 3474, loss 0.157211, acc 0.94
2018-04-21T23:53:04.242838: step 3475, loss 0.2941, acc 0.9
2018-04-21T23:53:04.288786: step 3476, loss 0.33334, acc 0.82
2018-04-21T23:53:04.334657: step 3477, loss 0.203326, acc 0.88
2018-04-21T23:53:04.380689: step 3478, loss 0.228637, acc 0.88
2018-04-21T23:53:04.426000: step 3479, loss 0.21982, acc 0.9
2018-04-21T23:53:04.475810: step 3480, loss 0.105191, acc 0.96

2018-04-21T23:53:13.429372: step 3585, loss 0.091573, acc 1
2018-04-21T23:53:13.473857: step 3586, loss 0.263172, acc 0.94
2018-04-21T23:53:13.517798: step 3587, loss 0.136231, acc 0.96
2018-04-21T23:53:13.561801: step 3588, loss 0.136682, acc 0.94
2018-04-21T23:53:13.607617: step 3589, loss 0.178896, acc 0.92
2018-04-21T23:53:13.655669: step 3590, loss 0.226771, acc 0.94
2018-04-21T23:53:13.699474: step 3591, loss 0.242998, acc 0.9
2018-04-21T23:53:13.743665: step 3592, loss 0.25939, acc 0.84
2018-04-21T23:53:13.787775: step 3593, loss 0.249442, acc 0.92
2018-04-21T23:53:13.831458: step 3594, loss 0.195383, acc 0.96
2018-04-21T23:53:13.878051: step 3595, loss 0.19766, acc 0.9
2018-04-21T23:53:13.922768: step 3596, loss 0.139018, acc 0.96
2018-04-21T23:53:13.967197: step 3597, loss 0.120305, acc 0.96
2018-04-21T23:53:14.011148: step 3598, loss 0.14206, acc 0.96
2018-04-21T23:53:14.054770: step 3599, loss 0.25943, acc 0.88
2018-04-21T23:53:14.101944: step 3600, loss 0.267607, acc 0.86



2018-04-21T23:53:22.917236: step 3704, loss 0.441775, acc 0.84
2018-04-21T23:53:22.963432: step 3705, loss 0.313205, acc 0.86
2018-04-21T23:53:23.008414: step 3706, loss 0.243189, acc 0.92
2018-04-21T23:53:23.053206: step 3707, loss 0.127812, acc 0.96
2018-04-21T23:53:23.098090: step 3708, loss 0.214786, acc 0.94
2018-04-21T23:53:23.145932: step 3709, loss 0.190671, acc 0.9
2018-04-21T23:53:23.189686: step 3710, loss 0.280958, acc 0.88
2018-04-21T23:53:23.234014: step 3711, loss 0.161961, acc 0.92
2018-04-21T23:53:23.277865: step 3712, loss 0.0891755, acc 0.98
2018-04-21T23:53:23.322530: step 3713, loss 0.186829, acc 0.92
2018-04-21T23:53:23.370333: step 3714, loss 0.193074, acc 0.94
2018-04-21T23:53:23.415698: step 3715, loss 0.410153, acc 0.88
2018-04-21T23:53:23.460419: step 3716, loss 0.210701, acc 0.9
2018-04-21T23:53:23.505667: step 3717, loss 0.184105, acc 0.92
2018-04-21T23:53:23.549721: step 3718, loss 0.208357, acc 0.94
2018-04-21T23:53:23.599149: step 3719, loss 0.141072, ac

2018-04-21T23:53:32.654853: step 3825, loss 0.0754596, acc 0.96
2018-04-21T23:53:32.699633: step 3826, loss 0.201616, acc 0.94
2018-04-21T23:53:32.744312: step 3827, loss 0.116933, acc 0.96
2018-04-21T23:53:32.789160: step 3828, loss 0.135002, acc 0.96
2018-04-21T23:53:32.834511: step 3829, loss 0.222897, acc 0.9
2018-04-21T23:53:32.885705: step 3830, loss 0.335654, acc 0.9
2018-04-21T23:53:32.932187: step 3831, loss 0.205687, acc 0.94
2018-04-21T23:53:32.977981: step 3832, loss 0.137586, acc 0.94
2018-04-21T23:53:33.023685: step 3833, loss 0.19487, acc 0.92
2018-04-21T23:53:33.068982: step 3834, loss 0.155586, acc 0.94
2018-04-21T23:53:33.118939: step 3835, loss 0.14418, acc 0.94
2018-04-21T23:53:33.163668: step 3836, loss 0.18517, acc 0.94
2018-04-21T23:53:33.208627: step 3837, loss 0.0846846, acc 0.96
2018-04-21T23:53:33.253774: step 3838, loss 0.430781, acc 0.92
2018-04-21T23:53:33.298144: step 3839, loss 0.143628, acc 0.92
2018-04-21T23:53:33.347610: step 3840, loss 0.138694, acc 